
# Streamlit版を「ジュピター形式」に置き換えた RAG 学習ノートブック  
**LangChain + OpenAI + Chroma（＋BM25ハイブリッド）**

このノートブックは、元の Streamlit アプリの処理フローを **講義用に段階分解** したものです。  
各セルで「何をしているか／なぜ必要か」を説明し、**最終的に同じロジック**（クエリ整形 → ハイブリッド検索 → 生成回答）をノートブック上で再現できるように構成しています。

> ⚠️ 実行には OpenAI API Key が必要です（有料課金が発生します）。`.env` ファイルまたは環境変数に設定してからご利用ください。



## 学習目標
- RAG 構成要素（**分割 → 埋め込み → ベクトルDB（Chroma） → キーワード（BM25） → ハイブリッド検索 → LLM整形**）を理解する  
- **社内RAG**を想定した ACL（閲覧権限）や「一般回答フォールバック」の考え方を学ぶ  
- **クエリ整形（凝縮・言い換え）**と **根拠付き回答**（引用リスト生成）の実装を追体験する


# 📘 RAG（Retrieval-Augmented Generation）入門

---

## 🔍 RAGとは？
RAG は **外部データベースから情報を検索（Retrieve）し、その情報を根拠に生成（Generate）する仕組み** です。  
単なる生成AI（ChatGPT等）と違い、**手元のドキュメントに基づく回答**が可能です。

---

## ✨ RAGを使うメリット
| 観点 | 従来のLLM | RAGを使ったLLM |
|------|-----------|----------------|
| **知識の鮮度** | 訓練データ時点までの知識に依存 | 自分で用意した最新ドキュメントを検索して利用可能 |
| **専門性** | 汎用知識は得意だが、社内固有情報は弱い | 内部資料やルールを検索 → 高精度に回答 |
| **再現性** | 同じ質問でも答えがぶれる | 根拠（context）があるため一貫性が増す |
| **透明性** | 出典が不明 | 参照元を明示可能（引用付き回答） |

---

## ⚙️ RAGの基本フロー

```mermaid
flowchart LR
    A[ユーザーの質問] --> B[クエリ変換/前処理]
    B --> C[ドキュメント検索<br>(Vector DB / BM25)]
    C --> D[関連文書群（context）]
    D --> E[LLMに渡す]
    E --> F[回答生成<br>+ 引用]


# RAGの精度向上のポイント

## ❓ なぜRAGは「惜しい答え」を返すのか
- 質問の曖昧さや社内用語で検索がズレる  
- チャンク分割やインデックス設計が不十分  
- 埋め込みや更新が古く、情報が正しく反映されていない  

---

## 🔑 RAGでの改善アプローチ方法
| 課題 | 改善の方向性 |
|------|--------------|
| 質問と検索結果がズレる | **クエリリライト** や意味理解による検索強化 |
| インデックス設計が不十分 | **適切なチャンク設計**（粒度の調整） |
| 埋め込みの鮮度不足 | **最新Embeddingの適用**と定期的な再構築 |
| 文書検索の網羅性不足 | **複数Retrieverの併用**（ベクトル＋BM25など） |
| 回答の根拠が不透明 | **引用付き回答** で信頼性を担保 |

---

##  6つの設計アプローチ（代表例）
1. **クエリリライト**: 曖昧な質問をLLMで意図に即した検索クエリへ変換  
2. **チャンク最適化**: 文書分割の大きさや重複範囲をチューニング  
3. **高品質Embedding**: より精度の高いモデルを利用  
4. **Retrieverの多様化**: ベクトル検索＋BM25のハイブリッド  
5. **フィルタリング**: ユーザー権限やドメインに応じた情報制御  
6. **運用設計**: インデックス更新や品質管理の仕組みを定着  

---

## 📊 まとめ
RAGの精度向上は「モデルの性能」だけでなく、  
**設計（チャンク・クエリ処理・Retriever選択）と運用** がカギとなる。  



## 0. 依存ライブラリのインストール（必要な場合）

> 既に環境が整っていれば **スキップ**して構いません。Google Colab などでは下を実行してください。


In [ ]:

# !pip install -U langchain langchain-community langchain-openai chromadb python-dotenv rank_bm25 unstructured
# ↑ BM25 は langchain_community の BM25Retriever を利用します（入らない環境では自動的にスキップする実装にしています）。



## 1. インポート
ipynbファイルでは Streamlit を外し、**ノートブックで読みやすい形**に再構成します。


In [1]:
# ============================================
# 目的：
#   RAG（Retrieval-Augmented Generation）で使う
#   ライブラリ・設定を読み込み、BM25 の有無を確認する最小例
#   ※「どの部品が何をするのか」を理解するための導入コード
# ============================================

import os
import re
import hashlib
import shutil
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

# --------------------------------------------
# .env から API キーなどの環境変数を読み込むためのユーティリティ
#   - .env に OPENAI_API_KEY="sk-..." のように書いておく
#   - 本番コードでは load_dotenv(); os.getenv("OPENAI_API_KEY") と組み合わせる
# --------------------------------------------
from dotenv import load_dotenv

# --------------------------------------------
# LangChain / RAG の主要部品
#  - Document: 1つの文書（本文＋メタデータ）を表現
#  - ChatPromptTemplate: LLM へのプロンプトをテンプレ化
#  - RecursiveCharacterTextSplitter: 長文を「チャンク」に分割
#  - TextLoader: .txt などのファイルを読み込んで Document に変換
#  - Chroma: ベクトルDB（埋め込みで検索するデータベース）
#  - ChatOpenAI / OpenAIEmbeddings: OpenAI の会話モデル／埋め込みモデル
# --------------------------------------------
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# --------------------------------------------
# BM25 は「キーワード一致型」の古典的検索アルゴリズム
#  - ベクトル検索（意味検索）と相性がよく、固有名詞などに強い
#  - LangChain では BM25Retriever として提供
#  - 内部で 'rank-bm25' ライブラリを利用するため、未インストールでも動くように try/except で吸収
#    → 入っていれば使う（HAS_BM25=True）、入ってなければスキップ（HAS_BM25=False）
# --------------------------------------------
try:
    from langchain_community.retrievers import BM25Retriever
    HAS_BM25 = True
except Exception:
    BM25Retriever = None  # 型ヒント用（IDE 補完のために None を入れておく）
    HAS_BM25 = False

# 実行時に BM25 が使えるかを出力（学習用デバッグ）
print("HAS_BM25:", HAS_BM25)


HAS_BM25: True



## 2. 定数とディレクトリ
- 埋め込みモデル：`text-embedding-3-small`（1536次元）
- 既存のベクトルDBを使い回す場合は **同じ次元**である必要があります


In [5]:

# --------------------------------------------
# プロジェクト内の主要フォルダ・設定を定義
# --------------------------------------------

# BASE_DIR: プロジェクトのルートディレクトリ
#   → 実行ファイルが置かれている場所を基準にする
BASE_DIR = Path(".").resolve()

# RES_DIR: 学習対象となるテキストファイルを置くフォルダ
#   例: resources/tech0_travel_policy.txt
RES_DIR = BASE_DIR / "resources"

# VECTOR_DIR: ベクトルデータベース（Chroma）の保存先フォルダ
#   "text-embedding-3-small" モデルを使うので、その名前を含めてフォルダ名を作成
VECTOR_DIR = RES_DIR / "note_text_embedding_3_small"

# PERSIST_DIR: Chroma に渡すときは Path ではなく str 型が必要なので変換
PERSIST_DIR = str(VECTOR_DIR)

# --------------------------------------------
# モデルや検索のパラメータ設定
# --------------------------------------------

# EMBED_MODEL: OpenAI の埋め込みモデル
#   - "text-embedding-3-small" は 1536 次元ベクトルを出力
#   - 他モデルに変える場合は保存先 (VECTOR_DIR) も変える必要あり
EMBED_MODEL = "text-embedding-3-small"  # 1536 dims

# CHUNK_SIZE: 1つの文書を分割するサイズ（文字数）
# CHUNK_OVERLAP: チャンク間で重複させる文字数
#   → 検索精度を高めるために前後を少しかぶらせる
CHUNK_SIZE = 900
CHUNK_OVERLAP = 180

# K_VECT: ベクトル検索で上位何件を取るか
# K_BM25: BM25検索で上位何件を取るか
# REL_THRESHOLD: 検索結果を有効とみなす最小の関連度スコア
K_VECT = 20
K_BM25 = 10
REL_THRESHOLD = 0.28

# ACL_CHOICES: アクセス制御リストの選択肢
#   → どのユーザーがどの文書を閲覧できるかを制御するためのタグ
ACL_CHOICES = ["public", "finance", "engineering", "sales"]

# --------------------------------------------
# 必要なフォルダを自動作成
#   exist_ok=True なので、既に存在していてもエラーにならない
# --------------------------------------------
RES_DIR.mkdir(parents=True, exist_ok=True)



## 3. .env / モデル設定
`.env` に `OPENAI_API_KEY=...` を入れておくか、環境変数に設定しておきます。


In [6]:

# --------------------------------------------
# 1. .env ファイルを読み込む
#   - .env 内に OPENAI_API_KEY="sk-..." を書いておく想定
#   - load_dotenv() を呼ぶと os.getenv で取得可能になる
# --------------------------------------------
load_dotenv()

# APIキーを環境変数から取得
api_key = os.getenv("OPENAI_API_KEY")

# APIキーが見つからなかった場合の警告
if not api_key:
    print("⚠️ OPENAI_API_KEY が見つかりません。`.env` または環境変数を設定してください。")

# --------------------------------------------
# 2. 使用する LLM モデルの指定
#   - 今回は、軽量版 "gpt-4.1-mini" などに変更することも可能
# --------------------------------------------
LLM_MODEL = "gpt-4.1-mini"

# --------------------------------------------
# 3. LLM インスタンスの準備
#   - ChatOpenAI: LangChain が提供する OpenAI API のラッパー
#   - api_key が未設定でもエラーで止まらないように
#     → None を代入して「後で設定されたら使える」設計にしている
# --------------------------------------------

# メインの対話モデル
llm = ChatOpenAI(api_key=api_key, model=LLM_MODEL) if api_key else None

# クエリ整形用の軽量モデル（例: 質問を短くまとめる）
rewriter = ChatOpenAI(api_key=api_key, model="gpt-4.1-mini") if api_key else None



## 4. UI ヘッダとサイドバーの設計意図

### ⚠️ 注意：このセルは Jupyter Notebook では必ずエラーになります ⚠️

以下のコードは **Streamlit 専用** の命令（`st.markdown`, `st.sidebar` など）を使っています。  

- Jupyter Notebook 上では Streamlit の UI コンポーネントは動作しないため、  
  実行すると必ず `NameError: name 'st' is not defined` になります。  

このセクションでは、RAG デモの 見た目と操作性 を担う UI を整えます。
狙いは次の 3 点です。

### RAGデモUIの設計要点

| 観点 | 内容 | 体験できるポイント |
|------|------|------------------|
| **情報設計の可視化** | ヘッダで「このアプリは RAG を体験する場所」と明示 | ユーザーが目的を誤解せず、体験の文脈を理解できる |
|  | 画面中央のイラストや紹介文で **社内限定知識（ミャクじぃ／旅費規程）** を提示 | RAGの「内部知識だけに答える」性質を直感的に伝える |
| **運用メニューの体験** | **ACL（権限）** 選択 | 「ユーザー属性によって見える情報が変わる」実務的要件を簡易再現 |
|  | フォールバック方針切替（a→c→b / a→b→c） | RAGが答えられないときの設計を比較体験できる |
|  | 一般回答の許可（allow_general）ON/OFF | OFFにすると **「根拠が無ければ答えない」** 安全設計を確認できる |
| **運用オペレーションの導線** | **インデックス再構築ボタン** | resources/ 配下を再読み込み → ベクトルDB再作成。現場での「更新運用」を模擬 |
|  | **チャット履歴クリア** | 会話状態を初期化 → 再テストしやすいUIを提供 |


### ⚠️ 注意：くどいですが、以下セルは Jupyter Notebook では必ずエラーになります ⚠️

In [7]:
# -----------------------------
# UIヘッダ
# -----------------------------
# --- 2段タイトルを HTML/CSS で描画 ---
#   ・Streamlit でも st.markdown(unsafe_allow_html=True) を使うと
#     シンプルな HTML/CSS を挿入できる
#   ・clamp() は画面幅に応じてフォントサイズを自動調整する関数
st.markdown("""
<style>
h1.app-title{
  font-weight: 800;           /* 太字で目立たせる */
  line-height: 1.15;          /* 行間 */
  margin: .2rem 0 .8rem;      /* 上右下左の余白 */
  /* 画面幅に応じてサイズ調整（最小2.2rem～最大3.6rem）*/
  font-size: clamp(2.2rem, 4vw + 1rem, 3.6rem);
}
/* 単語の途中で改行させたくない要素に付ける */
.nowrap{ white-space: nowrap; }
</style>

<h1 class="app-title">
  Streamlit RAG Starter <br/>
  <span class="nowrap">— LangChain + OpenAI + Chroma</span>
</h1>
""", unsafe_allow_html=True)

# 3カラムレイアウトを作り、中央カラム（幅比 1:2:1 の真ん中）に画像を置く
mid = st.columns([1, 2, 1])[1]
with mid:
    # HERO_IMG が存在する場合だけロゴ/キャラ画像を表示
    if HERO_IMG.exists():
        st.image(str(HERO_IMG), width=280)

# ヒーロー文言（RAGの主題を伝える）
#   ・Streamlit のテキストだけでは表現しづらい装飾を、HTMLで行う
#   ・unsafe_allow_html=True を付けると HTML が反映される
st.markdown(
    """
<div style="text-align:center; font-weight:700; margin: 8px 0 18px; font-size:1.05rem;">
私はミャクじぃ！世の中、だれも知らない架空のキャラクターじゃ。ChatGPTに聞いても、絶対にワシのことは知らないはず。<br>
このアプリにのみ、ミャクじぃに関する情報がRAGとして登録されている。もしRAGの凄さを体験したければ、ミャクじぃについて、質問するのじゃ！Tech0の旅費規程（架空）も登録しているぞ！
</div>
""",
    unsafe_allow_html=True,
)

# -----------------------------
# サイドバー（運用っぽい制御）
# -----------------------------
# 受講生が「設定を触りながら挙動を確かめる」ための UI 群。
# 実サービスでもよくある “運用メニュー” を想定。
st.sidebar.subheader("ボット設定")

# ・ユーザーの権限（ACL）を複数選択可能にする
#   → 文書メタデータの "acl" と突き合わせて表示/検索を制御する想定
user_acl = st.sidebar.multiselect("あなたの権限（ACL）", ACL_CHOICES, default=["public"])

# ・フォールバック戦略の選択
#   a: 明確化質問（Clarify）
#   b: 一般回答（General）
#   c: エスカレーション（Escalate）
#   「安全設計」は a→c→b の順、「使い勝手重視」は a→b→c の順で試す
mode = st.sidebar.selectbox("フォールバック方針", ["安全設計 (a→c→b)", "使い勝手重視 (a→b→c)"])

# ・RAGに根拠がないとき、一般知識で答えるのを許可するか（b）
#   学習時は OFF で「RAGだけだと答えられない」体験も見せると理解が深まる
allow_general = st.sidebar.toggle("RAGに無いとき一般回答も許可する（b）", value=False)
st.sidebar.caption("a: 明確化質問 / b: 一般回答 / c: エスカレーション")

# -----------------------------
# インデックス再構築ボタン
# -----------------------------
# 目的：
#   ・resources/ 配下のテキストを読み直し、ベクトルDB（Chroma）を作り直す
#   ・教材では「ファイルを追加/変更→押して更新」という運用フローを体験
if st.sidebar.button("🔁 インデックス再構築", help="resources配下を再読み込みしてベクトルDBを作り直します"):
    st.cache_resource.clear()                         # キャッシュをクリア
    shutil.rmtree(VECTOR_DIR, ignore_errors=True)     # 既存のベクトルDBを削除
    st.session_state["_reindexed"] = True             # フラグを立てる
    st.rerun()                                        # 画面をリロード（状態反映）

# 前段の処理直後に “再構築完了” を一度だけ表示するためのフラグ制御
if st.session_state.get("_reindexed"):
    st.sidebar.success("インデックスを再構築しました。")
    st.session_state["_reindexed"] = False

# -----------------------------
# チャット履歴のクリア
# -----------------------------
# 目的：
#   ・会話状態をリセットして、初期状態から試し直す
st.sidebar.divider()
if st.sidebar.button(
    "🧹 チャット履歴をクリア",
    use_container_width=True,
    help="会話ログを消して最初からやり直します"
):
    st.session_state.pop("messages", None)  # ← 履歴（セッション変数）を削除
    st.sidebar.success("履歴を消去しました。")
    st.rerun()  # 画面を再描画（ボタンのワンクリックで状態反映）


NameError: name 'st' is not defined


## 5. 読み込み & チャンク分割ユーティリティ
- **RecursiveCharacterTextSplitter** でチャンク化
- メタデータには `source / title / chunk / acl / domain` を付与

(ポイント)

「チャンク」とは？
長文を検索に適した大きさに切り分けた「小さな文書のかたまり」。
→ 900文字ごとに分け、前後を180文字かぶらせて検索精度を高める。

メタデータの役割
どのファイルの何番目か、アクセス権限(ACL)は何か、などを一緒に保存することで、後で「根拠付き回答」が可能になる。

拡張性
今は *.txt しか読んでいないが、*.md や *.pdf を追加できるように設計されている。


In [8]:
# --------------------------------------------
# ユーティリティ関数群
#   - RAG 用にリソースファイルを読み込む
#   - Chroma で扱える形に整形（チャンク分割＋メタデータ付与）
# --------------------------------------------

def _list_source_paths() -> List[Path]:
    """
    resources フォルダ内のテキストファイル一覧を返す関数。
    - 現状は *.txt のみ対象
    - 将来的には *.md や *.pdf も追加できる拡張性を残している
    """
    # Path.glob("*.txt") でファイル検索 → sorted で名前順に揃える
    return sorted((RES_DIR).glob("*.txt"))


def _sanitize_meta(meta: dict) -> dict:
    """
    メタデータを Chroma が受け付ける形式に変換する関数。
    - Chroma は str/int/float/bool/None のみ受け付ける仕様
    - list や dict などは文字列に変換して保存する
    """
    clean = {}
    for k, v in (meta or {}).items():
        if isinstance(v, (str, int, float, bool)) or v is None:
            clean[k] = v
        else:
            clean[k] = str(v)  # 受け付けない型は文字列に変換
    return clean


def _load_and_chunk(paths: List[Path]) -> List[Document]:
    """
    与えられたファイルパス群を読み込み、チャンク分割して Document のリストを返す。
    - RecursiveCharacterTextSplitter を使い、900字単位＋180字オーバーラップで分割
    - 各チャンクには「どのファイルの何番目か」が分かるメタデータを付与
    """
    # テキストをチャンクに切り分ける分割器を用意
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)

    out: List[Document] = []
    for p in paths:
        # 1. ファイル読み込み → Document リストを取得
        tdocs = TextLoader(str(p), encoding="utf-8").load()

        # 2. Document をチャンク分割
        splits = splitter.split_documents(tdocs)

        # 3. 各チャンクにメタデータを付与
        for i, d in enumerate(splits):
            d.metadata.update(
                _sanitize_meta(
                    {
                        "source": p.name,     # 元ファイル名（例: tech0_travel_policy.txt）
                        "title": p.stem,      # 拡張子なしファイル名（例: tech0_travel_policy）
                        "chunk": i,           # チャンク番号
                        "acl": "public",      # ACL: 今は固定値 "public"
                        "domain": "internal-policy",  # ドメイン分類（例: 社内規程）
                    }
                )
            )

        # 4. 加工済みチャンクを出力リストに追加
        out.extend(splits)

    return out



## 6. ベクトルストア & BM25 の構築（キャッシュ再利用あり）

この部分では、**検索の土台となるインデックス（索引）**を作成します。

【ベクトルストア（Chroma）】

OpenAI Embeddings で文書をベクトル化（意味を数値化）。
永続保存フォルダ（resources/note_text_embedding_3_small/）に格納。
既にフォルダが存在すれば再利用し、壊れていれば削除して再構築。

【BM25Retriever（オプション）】

古典的な キーワード一致検索。
固有名詞や Embeddings が苦手なクエリを補完する。
rank-bm25 がインストールされている場合のみ有効。

【返り値】

VS: ベクトルストア（Chroma インスタンス）
BM25: BM25Retriever（または None）
ALL_DOCS: すべてのチャンク化済みドキュメント

### build_indices() の返り値

| 返り値 | 型 | 役割 | イメージ / たとえ |
|--------|----|------|-------------------|
| **VS** | `Chroma` | 意味ベースの検索を行うベクトルデータベース | AI 司書が「意味」で似ている本を探してくれる |
| **BM25** | `BM25Retriever` または `None` | キーワードベースの検索を行う補完装置 | 図書カードを使って単語で探す昔ながらの検索 |
| **ALL_DOCS** | `List[Document]` | 検索対象となる全チャンク化済み文書 | 図書館の本棚に並んでいる全ての本 |



In [9]:
# --------------------------------------------
# ベクトルストア（Chroma）＋BM25 の構築関数
#   - ドキュメントを読み込み、埋め込みベクトル化して保存
#   - 既存DBがあれば再利用、無効なら再構築
#   - 追加で BM25Retriever も準備
# --------------------------------------------
def build_indices() -> Tuple[Any, Optional[Any], List[Document]]:
    """
    returns:
        VS        : ベクトルストア (Chroma)
        BM25      : BM25Retriever (オプション、無い場合は None)
        ALL_DOCS  : 読み込んだ全チャンク（Document のリスト）
    """

    # OpenAI の埋め込みモデルを初期化
    #  - ここで指定した model (1536次元) を使って Chroma に保存される
    embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"), model=EMBED_MODEL)

    # ----------------------------------------
    # 内部関数: リソースフォルダから読み直して再構築する処理
    # ----------------------------------------
    def rebuild() -> Tuple[Any, List[Document]]:
        paths = _list_source_paths()  # resources/*.txt を取得
        if not paths:
            raise FileNotFoundError(f"{RES_DIR} に *.txt がありません。")

        docs_ = _load_and_chunk(paths)  # チャンク化
        if not docs_:
            raise ValueError("読み込んだドキュメントが0件です。ファイル内容/文字コードを確認してください。")

        # Chroma に新規ベクトルDBを作成（persist_directory に保存される）
        vs_ = Chroma.from_documents(documents=docs_, embedding=embeddings, persist_directory=PERSIST_DIR)
        return vs_, docs_

    # ----------------------------------------
    # 既存のベクトルDBを使うか、再構築するかを判定
    # ----------------------------------------
    if VECTOR_DIR.exists():
        # 既存のDBを開く
        vs = Chroma(embedding_function=embeddings, persist_directory=PERSIST_DIR)
        try:
            # _collection.count() で保存件数を確認
            n = vs._collection.count()
        except Exception:
            n = 0

        if n == 0:
            # DBはあるが中身が壊れている → フォルダを削除して再構築
            shutil.rmtree(VECTOR_DIR, ignore_errors=True)
            vs, docs = rebuild()
        else:
            # 既存のコレクションを取得し、Document リストに変換
            raw = vs._collection.get(include=["metadatas", "documents"])
            docs = [
                Document(page_content=c, metadata=(m or {}))
                for c, m in zip(raw.get("documents", []), raw.get("metadatas", []))
            ]
            # もし Document が空なら、再構築
            if not docs:
                shutil.rmtree(VECTOR_DIR, ignore_errors=True)
                vs, docs = rebuild()
    else:
        # 初回起動 → フォルダが無いので再構築
        vs, docs = rebuild()

    # ----------------------------------------
    # BM25Retriever の準備（任意）
    #   - rank-bm25 がインストールされていれば有効
    #   - docs が存在する場合のみ初期化
    # ----------------------------------------
    bm25: Optional[Any] = None
    if HAS_BM25 and docs:
        try:
            bm25 = BM25Retriever.from_documents(docs)  # ドキュメント群からBM25インデックス作成
            bm25.k = K_BM25                            # 上位K件を返す設定
        except Exception as e:
            print("BM25 初期化に失敗しました（スキップします）:", e)
            bm25 = None

    return vs, bm25, docs


# --------------------------------------------
# 実行例
#   - APIキーが設定されていないとエラーになる
#   - 成功すれば Chroma のチャンク件数を出力
# --------------------------------------------
try:
    VS, BM25, ALL_DOCS = build_indices()
    print("Chroma OK / Chunks:", len(ALL_DOCS))
except Exception as e:
    VS = BM25 = ALL_DOCS = None
    print("⚠️ ベクトルストア構築に失敗しました:", e)


C:\Users\diamo\AppData\Local\Temp\ipykernel_6332\2685002047.py:40: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vs = Chroma(embedding_function=embeddings, persist_directory=PERSIST_DIR)
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


Chroma OK / Chunks: 5



## 7. 検索系（クエリ整形 / ルーティング）
- **domain_router**：内部／一般のルーティング
- **condense_query**：履歴からクエリを1行に凝縮（固有名詞は保持）
- **expand_queries**：言い換えを数種類作ってハイブリッド検索の当たりを増やす

### クエリ処理ユーティリティ

ユーザーの質問をそのまま検索に使うと、「曖昧すぎる」「ノイズが多い」といった問題が出ます。  
そこで、以下の4つの関数を用意して、**質問を検索に適した形に変換**します。

| 関数名 | 役割 | イメージ |
|--------|------|----------|
| `domain_router` | 質問の種類を分類（internal / both / general） | 「これは社内固有の話か？一般的な話か？」を仕分ける係 |
| `normalize_query` | クエリを正規化（空白や全角スペースを調整） | 入力のゆらぎを揃えて、検索精度を安定させる |
| `condense_query` | 履歴を踏まえて短くまとめ直す（固有名詞は残す） | 「長いやりとりを1文に整理する秘書」 |
| `expand_queries` | 言い換えを生成して複数クエリに展開 | 「別の言い方でも探してみよう」と検索の抜け漏れを防ぐ |

> **ポイント**  
> - `condense_query` と `expand_queries` は LLM を活用するので「AIが裏で検索に強い形に変換している」と伝えるとわかりやすい。  
> - `domain_router` の分類結果によって「RAGで探すか / 一般知識で答えるか」が変わる。  
> - 固有名詞は絶対に消さないことが重要（社内知識は特にユニークだから）。  


In [10]:
# --------------------------------------------
# クエリ処理まわりのユーティリティ関数
#   - ユーザーの質問を分類・整形・拡張して検索に適した形にする
# --------------------------------------------

def domain_router(q: str) -> str:
    """
    質問 (query) が「どの領域に関するものか」を判定する関数。
    - internal_kw に含まれる語があれば「社内専用の質問」と判定
    - both_kw に含まれる語があれば「社内/一般どちらもありうる質問」と判定
    - それ以外は「一般的な質問」と判定
    """
    internal_kw = ["ミャクじぃ", "ひろじぃ", "RAG", "ベクタ", "Chroma", "講義", "デモ",
                   "Tech0", "旅費", "旅費規程", "出張", "宿泊", "日当"]
    if any(k.lower() in q.lower() for k in internal_kw):
        return "internal"  # 社内情報に関する質問
    both_kw = ["設計", "要件", "仕様", "社内", "手順", "規程"]
    if any(k in q for k in both_kw):
        return "both"      # 社内にも一般にも該当しそうな質問
    return "general"       # 上記以外は一般知識で答えられる質問


def normalize_query(q: str) -> str:
    """
    ユーザーの質問文を正規化する関数。
    - 全角スペースを半角に置換
    - 前後の空白を除去
    - 複数の空白を1つにまとめる
    """
    q = q.replace("　", " ").strip()
    return re.sub(r"\s+", " ", q)


def condense_query(history: List[Dict], q: str) -> str:
    """
    履歴を踏まえて質問を1文にまとめ直す関数。
    - 固有名詞（例: ミャクじぃ / Tech0）は削除・変換せずそのまま保持
    - rewriter (LLM) を使って、検索に適した短い文に整形
    - APIキーが未設定なら、そのまま質問を返す（オフライン学習用）
    """
    if rewriter is None:
        # APIキーが無い場合でも授業が進められるように設計
        return q

    # 履歴の直近6件を取り出して "ユーザー: ... / アシスタント: ..." の形に整形
    last = "\n".join([
        f"{'ユーザー' if m['role']=='user' else 'アシスタント'}: {m['content']}"
        for m in history[-6:]
    ])

    # プロンプト設計：
    #  - 固有名詞は必ず残す
    #  - 省略や言い換えはせず、短い日本語1文に整形
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "あなたは検索クエリ整形の専門家です。"
         "【絶対条件】固有名詞・社内用語（例: ミャクじぃ/Tech0 等）は必ずそのまま残す。"
         "省略・言い換え・削除はしない。日本語1文、短く。出力はクエリのみ。"),
        ("human", "履歴:\n{h}\n\n直近質問:\n{q}")
    ])

    out = rewriter.invoke(prompt.format_messages(h=last, q=q)).content.strip()
    return out or q  # 万一 LLM が空を返した場合は元の質問を返す


def expand_queries(q: str, original: str) -> List[str]:
    """
    質問クエリを「言い換え」して複数バリエーションを生成する関数。
    - 元のクエリ (original) を必ず保持（優先度が最も高い）
    - 特定の固有名詞（ミャクじぃ）が含まれる場合は手作業で補強パターンを追加
    - rewriter があれば LLM に依頼して3つの言い換えを生成
    - rewriter が無い場合は簡易的に2パターンを追加
    - 最後に重複を除去して最大4件に絞る
    """
    seeds = [original]  # まずは原文をリストに入れる（必ず残す）

    # 固有名詞「ミャク」が含まれていれば手作業で候補を追加
    if "ミャク" in original:
        seeds += ["ミャクじぃ とは", "ミャクじぃ キャラクター", "ミャクじぃ 説明"]

    if rewriter is None:
        # オフライン環境用：シンプルに元クエリ＋「とは」を追加
        seeds += [q, q + " とは"]
    else:
        # LLM に「言い換えを3つ」お願いする
        p = ChatPromptTemplate.from_messages([
            ("system", "与えたクエリに対し意味の異なる日本語言い換えを3つ、改行で。説明不要。"),
            ("human", "{q}")
        ])
        out = rewriter.invoke(p.format_messages(q=q)).content.strip()
        seeds += [s.strip(" ・-•\n") for s in out.splitlines() if s.strip()]

    # 重複を除去し、先頭から最大4件を返す
    seen, ret = set(), []
    for s in seeds:
        if s not in seen:
            seen.add(s)
            ret.append(s)
    return ret[:4]



## 8. ハイブリッド検索（ACLつき）
- まず **ベクトル検索**でしきい値以上を確保  
- その後 **BM25**（ある場合）で補強し、スコアを調整
### ハイブリッド検索の仕組み

ユーザーの質問に対して、**意味検索（ベクトル）**と**キーワード検索（BM25）**を組み合わせる関数です。

| 関数 | 役割 | イメージ |
|------|------|----------|
| `uniq_id` | 各ドキュメントに一意のIDを付与 | 「本棚の本に背表紙ラベルを貼る」 |
| `_allow` | ユーザー権限（ACL）を確認 | 「閲覧可能な本かチェック」 |
| `hybrid_retrieve` | ベクトル検索＋BM25検索の結果を統合し、スコア順に返す | 「AI司書が意味で探し、図書カードで単語でも探す → 上位を提示」 |

#### ポイント
- **ベクトル検索**  
  - 意味が似ている文書を探す（高次元ベクトルの近さ）。
- **BM25検索**  
  - 単語の出現頻度を基準に探す。固有名詞や略語に強い。
- **ACLチェック**  
  - 文書ごとに「公開 / finance / engineering …」などの属性を持ち、ユーザーが閲覧できるものだけを返す。
- **ハイブリッド**  
  - 2つの検索を組み合わせることで「意味にもキーワードにも強い検索」が実現できる。


In [11]:
# --------------------------------------------
# 検索ユーティリティ
#   - ドキュメントに一意のIDを付与
#   - ACLチェック（アクセス制御）
#   - ベクトル検索とBM25検索を組み合わせて結果を返す
# --------------------------------------------

def uniq_id(doc: Document) -> str:
    """
    1つの Document を一意に識別するIDを生成。
    - source（元ファイル名）＋chunk番号 をベースにする
    - 内容（先頭200文字）のMD5ハッシュを加えて衝突を防ぐ
    例: 'tech0_travel_policy-0-a1b2c3d4...'
    """
    key = f"{doc.metadata.get('source','')}-{doc.metadata.get('chunk','')}"
    return key + "-" + hashlib.md5(doc.page_content[:200].encode("utf-8")).hexdigest()


def _allow(meta_acl: Optional[str], user_acl_list: List[str]) -> bool:
    """
    ドキュメントのACL（アクセス権限）を確認。
    - meta_acl が None → 全員アクセス可能
    - それ以外 → ユーザーの権限リストに含まれているかを判定
    """
    if meta_acl is None:
        return True
    return str(meta_acl) in set(user_acl_list)


def hybrid_retrieve(queries: List[str], acl: List[str]) -> List[Tuple[Document, float, str]]:
    """
    ハイブリッド検索を行う関数。
    - ベクトル検索（Chroma）と BM25 検索の両方を実行
    - ACLを満たさない文書は除外
    - 結果をスコア順に並べ、最大8件を返す

    returns: [(Document, relevance_score, "vect|bm25"), ...]
    """
    if VS is None:
        raise RuntimeError("ベクトルストアが未初期化です（OpenAIキーや依存関係を確認）。")

    pool: Dict[str, Tuple[Document, float, str]] = {}

    # ---- ベクトル検索（意味ベース） ----
    # Chromaのfilter機能は使わず、Python側でACLをチェックする
    for q in queries:
        docs_scores = VS.similarity_search_with_relevance_scores(q, k=K_VECT)
        for d, s in docs_scores:
            if not _allow(d.metadata.get("acl"), acl):
                continue
            uid = uniq_id(d)
            # relevance_score がしきい値以上ならプールに追加
            if uid not in pool and s >= REL_THRESHOLD:
                pool[uid] = (d, float(s), "vect")

    # ---- BM25検索（キーワードベース、オプション） ----
    if BM25:
        for q in queries:
            for d in BM25.get_relevant_documents(q):
                if not _allow(d.metadata.get("acl"), acl):
                    continue
                uid = uniq_id(d)
                if uid not in pool:
                    # 初回 → 固定スコア0.30で追加
                    pool[uid] = (d, 0.30, "bm25")
                else:
                    # 既にある場合はスコアを上書き（上限0.40）
                    old = pool[uid]
                    pool[uid] = (old[0], max(old[1], 0.40), old[2])

    # ---- スコア降順に並べて上位8件を返す ----
    return sorted(pool.values(), key=lambda x: x[1], reverse=True)[:8]



## 9. プロンプト & 回答合成
- RAG回答：**context のみ**で答える（一般知識は混ぜない）
- 一般回答：ミャクじぃ等の社内固有情報を使わず、一般的に説明
- 最後に「参考: タイトル#チャンク…」形式で引用を並べる

### 回答生成とフォールバック戦略

この部分では、**検索結果をどう答えるか**、そして**答えられなかった場合どうするか**を設計します。

| 名前 | 役割 | ポイント |
|------|------|----------|
| `SYSTEM_RAG` / `PROMPT_RAG` | RAG回答専用のルール | 「与えられた context だけで答える」「根拠がなければ答えない」 |
| `SYSTEM_GENERAL` / `PROMPT_GENERAL` | 一般回答用のルール | 社外知識を使うときはこちら。固有の社内キャラ情報は使わない |
| `compose_with_citations` | 文書の内容を根拠に回答を作成し、引用を付ける | 回答本文＋最大スコア＋参考情報を返す |
| `confidence_check` | 回答の自己信頼度を0〜1で評価 | contextに十分根拠があるかを LLM に自己採点させる |
| `fallback_strategy` | RAGで答えられなかったときの方針 | 「Clarify（質問を具体化）」→「General（一般回答）」→「Escalate（窓口へ）」の順 |

#### ポリシー切り替え
- **安全設計 (a→c→b)**  
  Clarify → Escalate → General  
  → 「安易に一般回答せず、安全第一で進める」
- **使い勝手重視 (a→b→c)**  
  Clarify → General → Escalate  
  → 「ユーザー体験を優先、まず一般回答を試す」

> **まとめ**  
> - **RAG回答**: 根拠があるときだけ答える  
> - **Confidence Check**: 信頼度が低ければフォールバックへ  
> - **フォールバック**: ポリシーに従って Clarify / General / Escalate を実行  


In [12]:
# --------------------------------------------
# プロンプト定義（回答のルールを指定するシステムメッセージ）
# --------------------------------------------

SYSTEM_RAG = """あなたは企業内RAGアシスタントです。与えられた context だけを根拠に日本語で簡潔に答えてください。
- contextに十分な根拠がない場合は、推測せず「このRAGでは該当情報が見つかりませんでした。」と答えてください。
- 社外情報や一般知識を混ぜないでください（フォールバックは別で処理します）。
- 箇条書き歓迎。各主張の後に [1], [2] のように参照番号を付け、最後に「参考: タイトル#チャンク…」を列挙してください。
"""

# RAG専用のプロンプトテンプレート
PROMPT_RAG = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_RAG),                       # システムメッセージ（ルール）
        ("human", "context:\n{context}\n\n質問: {q}") # ユーザー質問＋RAGの根拠
    ]
)

SYSTEM_GENERAL = """あなたは有能な日本語アシスタントです。事実に基づき、簡潔かつ丁寧に回答してください。
ミャクじぃ（社内固有）の情報は使わず、一般的な説明のみで回答してください。必要なら注意点も添えてください。"""

# 一般回答用のプロンプトテンプレート
PROMPT_GENERAL = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_GENERAL),
        ("human", "{q}")
    ]
)

# --------------------------------------------
# コンテキスト付き回答の生成
# --------------------------------------------
def compose_with_citations(q: str, docs: List[Tuple[Document, float, str]]) -> Tuple[str, float, str]:
    """
    文書の内容（context）を根拠に回答を生成し、参照情報も付ける。
    returns: (回答本文, 最大関連度スコア, 引用一覧文字列)
    """
    if llm is None:
        raise RuntimeError("OpenAI LLM が未初期化です。APIキーを設定してください。")

    if not docs:
        return "", 0.0, ""

    refs, parts = [], []
    # [1] doc_content ... の形でコンテキストをまとめる
    for i, (d, s, src) in enumerate(docs, start=1):
        parts.append(f"[{i}] {d.page_content}")
        title = d.metadata.get("title", d.metadata.get("source", "doc"))
        refs.append(f"[{i}] {title}#{d.metadata.get('chunk','')}")

    context = "\n\n".join(parts)
    msgs = PROMPT_RAG.format_messages(context=context, q=q)

    # LLMに投げて回答を生成
    out = llm.invoke(msgs).content

    # 最大関連度スコアを計算
    max_rel = max(score for _, score, _ in docs)

    return out, float(max_rel), "参考: " + ", ".join(refs)


# --------------------------------------------
# 回答の自己信頼度チェック
# --------------------------------------------
def confidence_check(text: str) -> float:
    """
    LLMに「この回答は社内contextだけで裏付けられているか」を判定させる。
    - 出力は0〜1の数値
    - 失敗時はデフォルト0.5
    """
    if llm is None:
        return 0.5

    p = ChatPromptTemplate.from_messages(
        [
            ("system", "あなたは回答の自己評価器です。以下の回答が、与えられた社内コンテキストだけで十分に裏付けられているかを0〜1で出力。数値のみ。"),
            ("human", "{ans}"),
        ]
    )
    try:
        s = llm.invoke(p.format_messages(ans=text)).content.strip()
        m = re.findall(r"1(?:\.0+)?|0\.\d+|0", s)  # 正規表現で数値を抽出
        return max(0.0, min(1.0, float(m[0]))) if m else 0.5
    except Exception:
        return 0.5


# --------------------------------------------
# フォールバック戦略
# --------------------------------------------
def fallback_strategy(q: str, policy: str, allow_general_flag: bool) -> str:
    """
    RAGで答えられなかった場合の対応方針を実装。
    - clarify: 質問をもっと具体化してもらう
    - escalate: 社内窓口にエスカレーション
    - general: 一般知識で回答する（許可されている場合のみ）
    """
    clarify = "もう少し具体的に教えてください。（例：対象範囲／時点／キーワード）"
    escalate = "社内窓口にエスカレーションしてください（例：#help-desk / 担当: support@example.com）。"

    if llm is None:
        general = "（一般回答は API キー未設定のため省略）"
    else:
        general = (
            llm.invoke(PROMPT_GENERAL.format_messages(q=q)).content
            if allow_general_flag
            else "一般回答は現在オフになっています（設定で有効化できます）。"
        )

    # ポリシーによって順序を変える
    if policy.startswith("安全設計"):
        # 安全性優先： Clarify → Escalate → General
        return f"{clarify}\n\n{escalate}\n\n{general if allow_general_flag else ''}".strip()
    else:
        # 使い勝手優先： Clarify → General → Escalate
        return f"{clarify}\n\n{general}\n\n{escalate}".strip()



## 10. 一問一答関数 `ask()`
- ルーティング → 凝縮 → 言い換え → ハイブリッド検索 → RAG回答  
- 十分な根拠がなければ **フォールバック**（明確化・一般回答・エスカレーション）

### ask() 関数の役割

ユーザーからの質問を受け取り、**RAG検索 → 信頼度チェック → フォールバック戦略**の流れで回答を返します。

#### 処理の流れ
1. **質問の分類と整形**
   - `domain_router` で internal / both / general に分類
   - `normalize_query` で正規化
   - `condense_query` で短い検索クエリに変換
   - `expand_queries` で言い換えクエリを追加

2. **RAG検索（社内ドメインのみ）**
   - `hybrid_retrieve` で Chroma（ベクトル検索）＋BM25（キーワード検索）
   - `compose_with_citations` で参照付き回答を作成
   - `confidence_check` で回答の信頼度を算出

3. **フォールバック処理**
   - 回答が無い / 信頼度が低い場合は、ポリシーに応じて  
     - Clarify（質問を具体化依頼）  
     - General（一般知識で回答、許可時のみ）  
     - Escalate（社内窓口にエスカレーション）  

4. **最終回答を組み立て**
   - 引用情報を本文に追加
   - 辞書形式で返却

#### 戻り値
| キー | 内容 |
|------|------|
| `answer` | 最終回答文 |
| `citations` | 引用情報（「参考: …」） |
| `confidence` | 信頼度スコア（0〜1） |
| `queries_used` | 実際に使われた検索クエリ一覧 |
| `domain` | 質問の分類結果（internal / both / general） |

---

👉 この関数は「RAGアプリの司令塔」。  
質問が来たときに **どの処理を実行し、どのルートで答えるかを決める心臓部**です。


In [13]:
# --------------------------------------------
# ask(): ユーザーからの質問を受け取り、RAG＋フォールバックで回答を生成する関数
# --------------------------------------------

def ask(q_raw: str,
        user_acl: Optional[List[str]] = None,
        policy_mode: str = "安全設計 (a→c→b)",
        allow_general: bool = False,
        history: Optional[List[Dict]] = None) -> Dict[str, Any]:
    """
    Returns: 辞書形式で結果を返す
      {
        'answer': str,         # 回答本文
        'citations': str,      # 引用情報（「参考: …」）
        'confidence': float,   # 信頼度スコア（0〜1）
        'queries_used': List[str], # 実際に使ったクエリ群
        'domain': str          # 質問の分類（internal/both/general）
      }
    """

    # ----------------------------------------
    # 事前処理（ACLと履歴）
    # ----------------------------------------
    user_acl = user_acl or ["public"]   # デフォルト権限は "public"
    history = history or []             # 履歴が無ければ空リスト

    # ----------------------------------------
    # 質問の整形と分類
    # ----------------------------------------
    domain = domain_router(q_raw)                       # 質問が社内か一般かを判定
    q_norm = normalize_query(q_raw)                     # 空白などを正規化
    q_condensed = condense_query(history, q_norm)       # 履歴を踏まえて短く整形
    queries = expand_queries(q_condensed, original=q_norm)  # 言い換えを含めたクエリ群

    # ----------------------------------------
    # 初期値の準備
    # ----------------------------------------
    answer_text = ""   # 最終的な回答
    citations = ""     # 参照情報
    conf = 0.0         # 信頼度

    # ----------------------------------------
    # RAG検索（社内関連ドメインの質問のみ）
    # ----------------------------------------
    if domain in ("internal", "both"):
        docs = hybrid_retrieve(queries, acl=user_acl)
        if docs:
            # 参照付き回答を作成
            answer_text, relmax, citations = compose_with_citations(q_norm, docs)

            # LLMによる自己評価と、検索スコアの平均を信頼度として利用
            conf_llm = confidence_check(answer_text)
            conf = (relmax + conf_llm) / 2.0

    # ----------------------------------------
    # フォールバック処理
    # ----------------------------------------
    if not answer_text or conf < 0.35:   # 回答が無い or 信頼度が低い場合
        if domain in ("general", "both"):   # 一般質問の可能性あり
            if allow_general:
                if llm is None:
                    answer_text = "（一般回答は API キー未設定のため省略）"
                else:
                    # 一般知識回答を LLM に依頼
                    answer_text = llm.invoke(
                        PROMPT_GENERAL.format_messages(q=q_norm)
                    ).content
            else:
                # 一般回答は禁止 → Clarify/Escalate のみ
                answer_text = fallback_strategy(q_norm, policy=policy_mode, allow_general_flag=allow_general)
        else:
            # internal のみだが回答不可 → Clarify/Escalateへ
            answer_text = fallback_strategy(q_norm, policy=policy_mode, allow_general_flag=allow_general)

    # ----------------------------------------
    # 引用を本文に追記
    # ----------------------------------------
    if citations:
        answer_text = f"{answer_text}\n\n{citations}"

    # ----------------------------------------
    # 辞書形式で返却
    # ----------------------------------------
    return {
        "answer": answer_text,
        "citations": citations,
        "confidence": conf,
        "queries_used": queries,
        "domain": domain,
    }



## 11. デモ実行

### ⚠️ 注意：このセルは 上から順番に実行していただければ動きます（第４章を除く） ⚠️

必要に応じて質問文を変えて実行してください。  
（API キー未設定の環境では、ベクトルストア構築や LLM 呼び出しが失敗する場合があります）

### 実行例: 「ミャクじぃについて教えて」

ここでは実際に `ask()` を呼び出して動作を確認します。

#### 入力
- **質問**: 「ミャクじぃについて教えて」
- **ACL**: public
- **フォールバック方針**: 安全設計 (a→c→b)
- **一般回答**: 無効（allow_general=False）

#### 出力される項目
| 項目 | 内容 |
|------|------|
| `domain` | internal / both / general のどれに分類されたか |
| `queries_used` | 実際に検索に使ったクエリのリスト |
| `answer` | 生成された回答本文 |
| `confidence` | 回答の信頼度スコア（0〜1） |

#### ポイント
- **domain** が `internal` になる → 「ミャクじぃ」は社内固有ワードと判定されたため。  
- **queries_used** にはオリジナル質問＋言い換えクエリが含まれる。  
- **answer** は RAG から見つかった情報に基づいて回答。根拠が無ければ「該当情報が見つかりませんでした」と返す。  
- **confidence** は RAG の検索スコアと自己評価の平均。0.35未満ならフォールバック戦略に移行する。  

> この例では「社内専用キャラクターについて質問 → RAGが回答する」流れを実演しています。



In [14]:
# --------------------------------------------
# デモ実行例: ask() を使って質問に答えさせる
# --------------------------------------------

# 履歴（会話コンテキスト）を保存するリスト
# ここでは空で開始。ユーザーのやり取りを順番に追加していく。
history = []  
# 例: [{'role': 'user', 'content': 'ミャクじぃってだれ？'}]

# 実際の質問例
# 社内固有ワード「ミャクじぃ」を含むため domain_router は internal に振り分ける想定
question = "ミャクじぃについて教えて"

try:
    # ask() を呼び出して回答を取得
    result = ask(
        question,
        user_acl=["public"],                    # ユーザー権限（ここでは public）
        policy_mode="安全設計 (a→c→b)",         # フォールバック方針
        allow_general=False,                    # 一般回答は許可しない
        history=history                         # 会話履歴を渡す
    )

    # 結果を出力
    print("Domain:", result["domain"])             # internal / both / general
    print("Queries:", result["queries_used"])      # 実際に検索に使ったクエリ
    print("--- Answer ---")
    print(result["answer"][:2000])                 # 回答本文（長い場合は先頭2000文字）
    print("\nConfidence:", result["confidence"])   # 信頼度スコア

except Exception as e:
    # 例外が起きた場合はエラーメッセージを表示
    print("実行時エラー:", e)


Number of requested results 20 is greater than number of elements in index 5, updating n_results = 5
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
C:\Users\diamo\AppData\Local\Temp\ipykernel_6332\3474795112.py:47: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'acl': 'public', 'chunk': 1, 'domain': 'internal-policy', 'source': 'rag.txt', 'title': 'rag'}, page_content='見た目は独特な赤と青をベースにしつつ、ひろじぃの象徴である眼鏡とほんの少しの白髪混じりの眉毛が加わっている。さらに、胸元には「Python」のロゴが脈打つように光っており、手にはノートPCを抱えている。時折そのノートPCは、ラッピング新幹線やJALミャクじぃJETに変形し、最新技術と夢を乗せて未来へ飛び立つという。\n\n命名の背景として、「ミャクじぃ」という名前は「命や知恵が脈々と続く」という意味を受け継ぎつつ、人生の後半戦を迎えたひろじぃの親しみやすさを重ね合わせたもの。大阪的なノリと親近感が強く、受講生や仲間からは「ミャクさん」「じぃさん」と愛称で呼ばれることもある。\n\n活動の舞台は万博にとどまらず、日常的にオンライン講義やワークショップにも登場し、「AIって何？」「Pythonって何ができるの？」といった質問にユーモラスに答える。エラー画面すら笑いに変える姿は、子どもから大人まで幅広い層に愛されている。また、グッズ展開も盛んで、ミャクじぃ型のUSBメモリや、エラー音が「じぃ〜」と鳴るスマホケースなど、奇抜ながらもユーモアあふれるアイテムが人気を博している。\n\n近年は、ラッピング電車や航空機にもミャクじぃが描かれ、「失敗してもええやん、学

Domain: internal
Queries: ['ミャクじぃについて教えて', 'ミャクじぃ とは', 'ミャクじぃ キャラクター', 'ミャクじぃ 説明']
--- Answer ---
ミャクじぃについては以下の通りです。

- ミャクじぃはTech0でITを学び始めた40歳の「ひろじぃ」から生まれたユーモラスな仮想キャラクターで、大阪・関西万博の公式キャラ「ミャクミャク」とは無関係です[2]。
- 見た目は赤と青をベースに眼鏡と白髪混じりの眉毛が特徴で、胸元には「Python」のロゴが光っています。手にはノートPCを持ち、時折それがラッピング新幹線やJALミャクじぃJETに変形します[1]。
- 名前の由来は「命や知恵が脈々と続く」意味と、ひろじぃの親しみやすさを重ねたもので、大阪的なノリと親近感が強く、仲間からは「ミャクさん」「じぃさん」と呼ばれています[1]。
- 性格は人懐っこくややおっちょこちょいで、プログラミングの失敗も楽しみながら仲間と共有し、失敗から学ぶことを体現しています[2]。
- 活動は万博だけでなくオンライン講義やワークショップにも広がり、幅広い層に愛されており、グッズ展開やラッピング電車・航空機にも登場しています[1]。
- 海外でもベトナムやオーストラリアでデジタル教育を伝える活動を行っています[1]。
- ミャクじぃは「金融からITへ挑戦する大人の学び」「万博の未来と命のつながり」「親しみやすいキャラ性」を併せ持ち、年齢に関係なく学び直しや挑戦ができることを象徴しています[3]。

参考: ミャクじぃの特徴と背景#1、誕生と性格#2、総合的意義#3

参考: [1] rag#1, [2] rag#0, [3] rag#2, [4] tech0_travel_policy#1, [5] tech0_travel_policy#0

Confidence: 0.7
